In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers
!pip install pytorch_lightning
!pip install multilingual-clip
!pip install langchain unstructured[all-docs] pydantic lxml

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-r3dsj7h9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-r3dsj7h9
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 807.0 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
%cd /content
!git clone -b v1.0 https://github.com/camenduru/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2
!pip install -e .

/content
Cloning into 'LLaVA'...
remote: Enumerating objects: 1960, done.
remote: Total 1960 (delta 0), reused 0 (delta 0), pack-reused 1960
Receiving objects: 100% (1960/1960), 13.60 MiB | 23.61 MiB/s, done.
Resolving deltas: 100% (1173/1173), done.
/content/LLaVA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.5 MB/s eta 0:00:00
Obtaining file:///content/LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 812.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.

In [ ]:
import warnings
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [ ]:
import torch
import clip
# !pip install git+https://github.com/openai/CLIP.git
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# clip_model, compose = clip.load('ViT-L/14', device = device)
# text_model = text_model.cpu()
# def process(idx_val,arr):
#   if idx_val=='0':
#     arr.append(0)
#   else:
#     arr.append(1)

cpu


In [ ]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import torch
import math
from torch import nn
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score,precision_score
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from PIL import Image
from torchvision import transforms
from PIL import ImageFile
import fireworks.client
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
data = pd.read_csv('/content/personification_train.csv')

In [ ]:
data.head(10)

,Name,text,persuasive_inten,Persuasive,personification,Irony,Alliteration,Analogies,Invective,Metaphor,puns_and_wordplays,Satire,Hyperboles
0,data0.png,Shivam Dube Lays company making chips* chips b...,3,1,1,0,0,0,0,0,0,0,1
1,data1.png,Kbhi bhi niyam badal sakta hai Sher ki umar jy...,0,0,0,0,0,0,0,0,0,0,0
2,data10.png,"Teacher. So , There phases of matter are Solid...",0,0,0,0,0,0,0,0,0,0,0
3,data100.png,Do it quick Before a non- Muslim comes in,1,1,0,1,0,0,0,0,0,0,0
4,data101.png,"Mujhe divorce chahiye, mere pati active nahi h...",2,1,0,1,0,1,0,0,0,0,0
5,data102.png,Yo Kejriwal so honest He got this memory card ...,4,1,0,0,0,1,0,0,0,0,0
6,data103.png,"#Aajse TumharaNaam Hello, Michael Jakson Speak...",2,1,0,0,0,0,0,0,1,1,0
7,data104.png,When guests/relatives visit your house:- Abhi ...,0,0,0,0,0,0,0,0,0,0,0
8,data105.png,"Ye Allah isse bhi aage hota hai kya hota hai, ...",2,1,0,0,0,0,0,0,1,0,0
9,data106.png,"Hume toh apno ne loota, gairo me kaha dam tha ...",0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from multilingual_clip import pt_multilingual_clip
import transformers

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2024-05-12 12:12:44,083] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
#just to check how multilingual clip works
texts = [
    'Three blind horses listening to Mozart.',
    'Älgen är skogens konung!',
    'Wie leben Eisbären in der Antarktis?',
    'Вы знали, что все белые медведи левши?'
]
model_name = 'M-CLIP/XLM-Roberta-Large-Vit-L-14'
# Load Model & Tokenizer
model = pt_multilingual_clip.MultilingualCLIP.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
embeddings = model.forward(texts, tokenizer)
print(embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

torch.Size([4, 768])


In [ ]:
sample = data['text'][10]
sample

"When your son's name is Allah Akbar And you lose him at the mall"

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = preprocess(Image.open('/content/drive/MyDrive/Gitanjali Mam/persuasive_meme/aloknath0.png')).unsqueeze(0).to(device)
text = clip.tokenize([sample]).to(device)

In [ ]:
text

tensor([[49406,   827,   695,   825,   568,  1981,   533,  6378, 27180,   537,
           592,  5354,  1269,   536,   518,  6220, 49407,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']='2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# clip_model, compose = clip.load('RN50x4', device = device)
clip_model, compose = clip.load("ViT-B/32", device = device)
text_inputs = (clip.tokenize(data.text.values[321],truncate=True)).to(device)
print(text_inputs)

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 76.8MiB/s]


tensor([[49406,   257,  1808, 14310,   530, 38281,  1035,   328,   267,  1808,
         14310,   530,  1586,   658,  1035,   328,   269,  1191,   625,  4582,
          7448, 21492,  1272,  1587,   677,  4582,  3823,  8135, 24797,   717,
          8981, 34206, 46985,  2697,   525,   787,  2767,  3048,   269,   320,
          3236,  2927,  2636,   977, 11950,   566,  9032,   269,   320,  5183,
          3569,  4432,  8171,   537,   320,  2328,   825,   539, 17430, 17270,
           269, 10889, 15537,   256, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)


In [ ]:
len(data)

4799

In [ ]:
data.Name.values[1:10]

array(['data1.png', 'data10.png', 'data100.png', 'data101.png',
       'data102.png', 'data103.png', 'data104.png', 'data105.png',
       'data106.png'], dtype=object)

In [ ]:
data.head()

,Name,text,persuasive_inten,Persuasive,personification,Irony,Alliteration,Analogies,Invective,Metaphor,puns_and_wordplays,Satire,Hyperboles
0,data0.png,Shivam Dube Lays company making chips* chips b...,3,1,1,0,0,0,0,0,0,0,1
1,data1.png,Kbhi bhi niyam badal sakta hai Sher ki umar jy...,0,0,0,0,0,0,0,0,0,0,0
2,data10.png,"Teacher. So , There phases of matter are Solid...",0,0,0,0,0,0,0,0,0,0,0
3,data100.png,Do it quick Before a non- Muslim comes in,1,1,0,1,0,0,0,0,0,0,0
4,data101.png,"Mujhe divorce chahiye, mere pati active nahi h...",2,1,0,1,0,1,0,0,0,0,0


In [ ]:
import fireworks.client
import os
fireworks.client.api_key = os.getenv("")
fireworks.client.api_key = ""

def get_completion(prompt, model=None, max_tokens=50):
    fw_model_dir = "accounts/fireworks/models/"
    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model
    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        n=1,
        max_tokens=200,
        temperature=0.1,
        top_p=0.9
    )
    return completion.choices[0].text

def get_prompt_text(text_re):
  prompt1 = f"Retrieve relevant information about the meme with the text transcription: {text_re}  Explain the sentiment with context and any additional insights associated with this meme"
  return prompt1

prompt1 = "Retrieve relevant information about the meme with the text transcription: Explain the sentiment with context and any additional insights associated with this meme"
mistral_llm = "mistral-7b-instruct-4k"
get_completion(prompt1, model=mistral_llm)

#get_completion(prompt1+ data['Text_Transcription'][10], model=mistral_llm)

'.\n\nContext: The meme is a picture of a person holding a sign that says "I\'m not a doctor, but I can tell you that wearing a mask is not a conspiracy theory." The person is wearing a mask and holding the sign in front of a group of people who are not wearing masks.\n\nSentiment: The sentiment of the meme is positive and informative. The person is trying to educate others about the importance of wearing masks during the COVID-19 pandemic. The meme is also meant to address the misinformation and conspiracy theories surrounding the use of masks.\n\nAdditional Insights: The meme is a powerful tool for spreading awareness and education about important public health issues. It is also a reminder that individuals have a responsibility to educate themselves and others about important topics, especially during times of crisis. The meme is also a reflection of the ongoing debate surrounding the use of masks during the pandemic, with some people believing that'

In [ ]:
data.head()

,Name,text,persuasive_inten,Persuasive,personification,Irony,Alliteration,Analogies,Invective,Metaphor,puns_and_wordplays,Satire,Hyperboles
0,data0.png,Shivam Dube Lays company making chips* chips b...,3,1,1,0,0,0,0,0,0,0,1
1,data1.png,Kbhi bhi niyam badal sakta hai Sher ki umar jy...,0,0,0,0,0,0,0,0,0,0,0
2,data10.png,"Teacher. So , There phases of matter are Solid...",0,0,0,0,0,0,0,0,0,0,0
3,data100.png,Do it quick Before a non- Muslim comes in,1,1,0,1,0,0,0,0,0,0,0
4,data101.png,"Mujhe divorce chahiye, mere pati active nahi h...",2,1,0,1,0,1,0,0,0,0,0


In [ ]:
def get_data(data):
  #data = pd.read_csv(dataset_path)
  text = list(data['text'])
  img_path = list(data['Name'])
  name = list(data['Name'])
  persuasive_inten = list(data['persuasive_inten'])
  #persuasive_inten = list(map(lambda x: x - 1 , persuasive_inten))
  label = list(data['Persuasive'])
  t3_1 =list(data['Irony'])
  t3_2 = list(data['personification'])
  t3_3 = list(data['Alliteration'])
  t3_4 = list(data['Analogies'])
  t3_5 = list(data['Invective'])
  t3_6 = list(data['Metaphor'])
  t3_7 = list(data['puns_and_wordplays'])
  t3_8 = list(data['Satire'])
  t3_9 = list(data['Hyperboles'])

  text_features,image_features,rag_features,Name,l,ir,per,alli,ana,inv,meta,puaps,sat,hyp,persi = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

  for txt,img,L,n,a,b,c,d,e,f,g,h,i,v in tqdm(zip(text,img_path,label,name,t3_1,t3_2,t3_3,t3_4,t3_5,t3_6,t3_7,t3_8,t3_9,persuasive_inten)):
    try:
      img = Image.open('/content/drive/MyDrive/Gitanjali Mam/persuasive_meme/'+img)
    except Exception as e:
      print(e)
      continue
    txt_rag = get_completion(get_prompt_text(txt)+txt, model=mistral_llm)
    txt2 = txt_rag
    img = torch.stack([compose(img).to(device)])
    l.append(L)
    Name.append(n)
    ir.append(a)
    persi.append(v)
    per.append(b)
    alli.append(c)
    ana.append(d)
    inv.append(e)
    meta.append(f)
    puaps.append(g)
    sat.append(h)
    hyp.append(i)
    with torch.no_grad():
      temp_rag=model.forward(txt2, tokenizer).detach().cpu().numpy()
      rag_features.append(temp_rag)
      temp_txt=model.forward(txt, tokenizer).detach().cpu().numpy()
      text_features.append(temp_txt)
      temp_img = clip_model.encode_image(img).detach().cpu().numpy()
      image_features.append(temp_img)
      del temp_txt
      del temp_img
      torch.cuda.empty_cache()
    del img
    torch.cuda.empty_cache()
  return text_features,rag_features,image_features,l,Name,ir,per,alli,ana,inv,meta,puaps,sat,hyp,persi


#Pre-Processing:
#Converts the opened image (img) to a PyTorch tensor and stacks it into a batch
#Uses CLIP to encode text into text_features & image to image_features
#CLIP Uses Zero-Shot Learning_

In [ ]:
#trial
text_features,rag_features,image_features,l,Name,ir,per,alli,ana,inv,meta,puaps,sat,hyp,persi = get_data(data.head(5))

0it [00:00, ?it/s]

In [ ]:
outliers = []
for names in tqdm(list(data['Name'])):
  #change the path according to your drive
  if not os.path.exists('/content/drive/MyDrive/Gitanjali Mam/persuasive_meme/'+names):
    outliers.append(names)

# data = data[~data['Name'].isin(outliers)]

  0%|          | 0/4799 [00:00<?, ?it/s]

In [ ]:
class HatefulDataset(Dataset):

  def __init__(self,data):
    self.t_f,self.r_f,self.i_f,self.label,self.name, self.t3_1, self.t3_2, self.t3_3, self.t3_4, self.t3_5, self.t3_6, self.t3_7, self.t3_8, self.t3_9,self.persuasive_inten = get_data(data)

    self.t_f = np.squeeze(np.asarray(self.t_f),axis=1)
    self.r_f = np.squeeze(np.asarray(self.r_f),axis=1)
    self.i_f = np.squeeze(np.asarray(self.i_f),axis=1)

  def __len__(self):
    return len(self.label)

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    #print(idx)
    name=self.name[idx]
    label = self.label[idx]
    persuasive_inten = self.persuasive_inten[idx]
    t3_1 = self.t3_1[idx]
    t3_2 = self.t3_2[idx]
    t3_3 = self.t3_1[idx]
    t3_4 = self.t3_4[idx]
    t3_5 = self.t3_5[idx]
    t3_6 = self.t3_6[idx]
    t3_7 = self.t3_7[idx]
    t3_8 = self.t3_8[idx]
    t3_9 = self.t3_9[idx]
    T = self.t_f[idx,:]
    R = self.r_f[idx,:]
    I = self.i_f[idx,:]

    sample = {'label':label,'processed_txt':T,'processed_rag':R,'processed_img':I,'name':name,'persuasive_inten':persuasive_inten,'irony':t3_1,'personification':t3_2,'Alliteration': t3_3,'Analogies':t3_4,
              'Invective':t3_5,'Metaphor':t3_5,'puns_and_wordplays': t3_7,'Satire':t3_8,'Hyperboles':t3_9 }
    return sample

#Dataset Class for easily maintaing pipeline

#Init -> Pre-Process from get_data and convert to numpy array
#Len -> Total Number of Sample
#Getitem -> Sample at given idx in a List

#Text & Image features -> RAG  Output goes through CLIP
# We get otuput from multimodal MFB
# apply attention mechanism  -> concatenate & apply softmax

In [ ]:
sample_dataset = HatefulDataset(data.head(4500))

In [ ]:
# torch.save(sample_dataset,'/content/persuasive_train50.pt')
sample_dataset_new= torch.load("/content/new4000.pt")
test= torch.load("/content/new4000.pt")
len(sample_dataset_new)

4000

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MFB(nn.Module):
    def __init__(self,img_feat_size, ques_feat_size, is_first, MFB_K, MFB_O, DROPOUT_R):
        super(MFB, self).__init__()
        #self.__C = __C
        self.MFB_K = MFB_K
        self.MFB_O = MFB_O
        self.DROPOUT_R = DROPOUT_R
        self.is_first = is_first
        self.proj_i = nn.Linear(img_feat_size, MFB_K * MFB_O)
        self.proj_q = nn.Linear(ques_feat_size, MFB_K * MFB_O)
        self.dropout = nn.Dropout(DROPOUT_R)
        self.pool = nn.AvgPool1d(MFB_K, stride = MFB_K)

    def forward(self, img_feat, ques_feat, exp_in=1):
        batch_size = img_feat.shape[0]
        img_feat = self.proj_i(img_feat)                # (N, C, K*O)
        ques_feat = self.proj_q(ques_feat)              # (N, 1, K*O)

        exp_out = img_feat * ques_feat             # (N, C, K*O)
        exp_out = self.dropout(exp_out) if self.is_first else self.dropout(exp_out * exp_in)     # (N, C, K*O)
        z = self.pool(exp_out) * self.MFB_K         # (N, C, O)
        z = torch.sqrt(F.relu(z)) - torch.sqrt(F.relu(-z))
        z = F.normalize(z.view(batch_size, -1))         # (N, C*O)
        z = z.view(batch_size, -1, self.MFB_O)      # (N, C, O)
        return z


#MFB -> Multimodal Factorized Bilinear Pooling
#used to model complex interactions between features like image and text
#MFB_K -> Number Of factors, MFB_O -> Output size,
#Init initializes linear projection layers for image and question features , dropout layer and average pooling layer

#Forward:

#exp_in = input expansion factor (default - 1)
#Linear projection of image and question features to factorized bilinear form
#Element-wise multiplication of image and question features
#APply Dropout
#Average pooling along the factorized dimension (MFB_K) to reduce the size of the output tensor
#Element-wise operations to compute the final output (z) using square root and normalization using Relu.
#The final output represents the fused representation of image and question features.


In [ ]:
data = data[~data['Name'].isin(outliers)]
len(sample_dataset_new)

4000

In [ ]:
# torch.manual_seed(123)
# t_p,v_p = torch.utils.data.random_split(sample_dataset_new,[450,50])

# # torch.manual_seed(123)
# t_p,te_p = torch.utils.data.random_split(t_p,[340,110])

In [ ]:
torch.manual_seed(123)
t_p,v_p = torch.utils.data.random_split(sample_dataset_new,[3500,500])

# torch.manual_seed(123)
t_p,te_p = torch.utils.data.random_split(t_p,[3000,500])

In [ ]:
t_p[1]["processed_img"].shape
t_p[1]['processed_txt'].shape
t_p[1]['processed_rag'].shape

(768,)

In [ ]:

class Classifier(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.MFB = MFB(512,768,True,256,64,0.1)
    self.fin_y_shape = torch.nn.Linear(768,512)
    self.fin_old = torch.nn.Linear(128,2)
    self.fin = torch.nn.Linear(16 * 768, 64)
    self.fin_inten = torch.nn.Linear(128,6)
    self.fin_e1 = torch.nn.Linear(128,2)
    self.fin_e2 = torch.nn.Linear(128,2)
    self.fin_e3 = torch.nn.Linear(128,2)
    self.fin_e4 = torch.nn.Linear(128,2)
    self.fin_e5 = torch.nn.Linear(128,2)
    self.fin_e6 = torch.nn.Linear(128,2)
    self.fin_e7 = torch.nn.Linear(128,2)
    self.fin_e8 = torch.nn.Linear(128,2)
    self.fin_e9 = torch.nn.Linear(128,2)
    self.validation_step_outputs = []
    self.test_step_outputs = []

  def forward(self, x,y,rag):
      x_,y_,rag_ = x,y,rag
      print("x.shape", x.shape)

      z = self.MFB(torch.unsqueeze(y, axis=1), torch.unsqueeze(x, axis=1))
      z_rag = self.MFB(torch.unsqueeze(y, axis=1), torch.unsqueeze(rag, axis=1))

      z_newe = torch.cat((z, z_rag), dim=2)
      z_new = torch.squeeze(z_newe, dim=1)
      c_inten = self.fin_inten(z_new)
      c_e1 = self.fin_e1(z_new)
      c_e2 = self.fin_e2(z_new)
      c_e3 = self.fin_e3(z_new)
      c_e4 = self.fin_e4(z_new)
      c_e5 = self.fin_e5(z_new)
      c_e6 = self.fin_e6(z_new)
      c_e7 = self.fin_e7(z_new)
      c_e8 = self.fin_e8(z_new)
      c_e9 = self.fin_e9(z_new)
      c = self.fin_old(z_new)

      output = torch.log_softmax(c, dim=1)
      c_inten = torch.log_softmax(c_inten, dim=1)
      c_e1 = torch.log_softmax(c_e1, dim=1)
      c_e2 = torch.log_softmax(c_e2, dim=1)
      c_e3 = torch.log_softmax(c_e3, dim=1)
      c_e4 = torch.log_softmax(c_e4, dim=1)
      c_e5 = torch.log_softmax(c_e5, dim=1)
      c_e6 = torch.log_softmax(c_e6, dim=1)
      c_e7 = torch.log_softmax(c_e7, dim=1)
      c_e8 = torch.log_softmax(c_e8, dim=1)
      c_e9 = torch.log_softmax(c_e9, dim=1)
      return output,c_inten,c_e1,c_e2,c_e3,c_e4,c_e5,c_e6,c_e7,c_e8,c_e9



  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
      lab,txt,rag,img,name,intensity,e1,e2,e3,e4,e5,e6,e7,e8,e9 = train_batch
      lab = train_batch[lab]
      #print(lab)
      txt = train_batch[txt]
      rag = train_batch[rag]
      img = train_batch[img]
      name= train_batch[name]
      intensity = train_batch[intensity]
      e1 = train_batch[e1]
      e2 = train_batch[e2]
      e3 = train_batch[e3]
      e4 = train_batch[e4]
      e5 = train_batch[e5]
      e6 = train_batch[e6]
      e7 = train_batch[e7]
      e8 = train_batch[e8]
      e9 = train_batch[e9]

      logit_offen,logit_inten_target,a,b,c,d,e,f,g,h,i= self.forward(txt,img,rag)

      loss1 = self.cross_entropy_loss(logit_offen, lab)
      loss4 = self.cross_entropy_loss(a, e1)
      loss5 = self.cross_entropy_loss(b, e2)
      loss6 = self.cross_entropy_loss(c, e3)
      loss7 = self.cross_entropy_loss(d, e4)
      loss8 = self.cross_entropy_loss(e, e5)
      loss9 = self.cross_entropy_loss(f, e6)
      loss10 = self.cross_entropy_loss(g, e7)
      loss11 = self.cross_entropy_loss(h, e8)
      loss12 = self.cross_entropy_loss(i, e9)
      loss17 = self.cross_entropy_loss(logit_inten_target, intensity)
      loss = loss1 + loss4 + loss5 + loss6 + loss7 + loss8 +loss9 + loss10 + loss11 + loss12 + loss17
      self.log('train_loss', loss)
      return loss

  def validation_step(self, val_batch, batch_idx):
      lab,txt,rag,img,name,intensity,e1,e2,e3,e4,e5,e6,e7,e8,e9= val_batch
      lab = val_batch[lab]
      txt = val_batch[txt]
      rag = val_batch[rag]
      img = val_batch[img]
      name = val_batch[name]
      intensity = val_batch[intensity]
      e1 = val_batch[e1]
      e2 = val_batch[e2]
      e3 = val_batch[e3]
      e4 = val_batch[e4]
      e5 = val_batch[e5]
      e6 = val_batch[e6]
      e7 = val_batch[e7]
      e8 = val_batch[e8]
      e9 = val_batch[e9]
      logits,inten,a,b,c,d,e,f,g,h,i = self.forward(txt,img,rag)
      logits=logits.float()
      tmp = np.argmax(logits.detach().cpu().numpy(),axis=1)
      loss = self.cross_entropy_loss(logits, lab)
      lab = lab.detach().cpu().numpy()
      self.log('val_acc', accuracy_score(lab,tmp))
      self.log('val_roc_auc',roc_auc_score(lab,tmp))
      self.log('val_loss', loss)
      tqdm_dict = {'val_acc': accuracy_score(lab,tmp)}
      self.validation_step_outputs.append({'progress_bar': tqdm_dict,'val_f1 offensive': f1_score(lab,tmp,average='macro')})

      return {
                'progress_bar': tqdm_dict,
      'val_f1 offensive': f1_score(lab,tmp,average='macro')
      }

  def on_validation_epoch_end(self):
    outs = []
    outs14=[]
    for out in self.validation_step_outputs:
       outs.append(out['progress_bar']['val_acc'])
       outs14.append(out['val_f1 offensive'])
    self.log('val_acc_all_offn', sum(outs)/len(outs))
    self.log('val_f1 offensive', sum(outs14)/len(outs14))
    print(f'***val_acc_all_offn at epoch end {sum(outs)/len(outs)}****')
    print(f'***val_f1 offensive at epoch end {sum(outs14)/len(outs14)}****')
    self.validation_step_outputs.clear()

  def test_step(self, batch, batch_idx):
      lab,txt,rag,img,name,intensity,e1,e2,e3,e4,e5,e6,e7,e8,e9= batch
      lab = batch[lab]
      txt = batch[txt]
      rag = batch[rag]
      img = batch[img]
      name = batch[name]
      intensity = batch[intensity]
      e1 = batch[e1]
      e2 = batch[e2]
      e3 = batch[e3]
      e4 = batch[e4]
      e5 = batch[e5]
      e6 = batch[e6]
      e7 = batch[e7]
      e8 = batch[e8]
      e9 = batch[e9]
      logits,inten,a,b,c,d,e,f,g,h,i= self.forward(txt,img,rag)
      logits = logits.float()
      tmp = np.argmax(logits.detach().cpu().numpy(force=True),axis=-1)
      loss = self.cross_entropy_loss(logits, lab)
      lab = lab.detach().cpu().numpy()
      self.log('test_acc', accuracy_score(lab,tmp))
      self.log('test_roc_auc',roc_auc_score(lab,tmp))
      self.log('test_loss', loss)
      tqdm_dict = {'test_acc': accuracy_score(lab,tmp)}
      self.test_step_outputs.append({'progress_bar': tqdm_dict,'test_acc': accuracy_score(lab,tmp), 'test_f1_score': f1_score(lab,tmp,average='macro')})
      return {
                'progress_bar': tqdm_dict,
                'test_acc': accuracy_score(lab,tmp),
                'test_f1_score': f1_score(lab,tmp,average='macro')
      }
  def on_test_epoch_end(self):
      outs = []
      outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14 = \
      [],[],[],[],[],[],[],[],[],[],[],[],[],[]
      for out in self.test_step_outputs:
        outs.append(out['test_acc'])
        outs2.append(out['test_f1_score'])
      self.log('test_acc', sum(outs)/len(outs))
      self.log('test_f1_score', sum(outs2)/len(outs2))
      self.test_step_outputs.clear()

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=5e-3)
    return optimizer


"""
Main Model:
Initialize
Forward Pass
Training Step
Validation Step
Testing Step

Pp
"""
class HmDataModule(pl.LightningDataModule):

  def setup(self, stage):
    self.hm_train = t_p
    self.hm_val = v_p
    # self.hm_test = test
    self.hm_test = te_p

  def train_dataloader(self):
    return DataLoader(self.hm_train, batch_size=64, drop_last=True)

  def val_dataloader(self):
    return DataLoader(self.hm_val, batch_size=64, drop_last=True)

  def test_dataloader(self):
    return DataLoader(self.hm_test, batch_size=128, drop_last=True)

data_module = HmDataModule()
checkpoint_callback = ModelCheckpoint(
     monitor='val_acc_all_offn',
     dirpath='mrinal/',
     filename='epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}',
     auto_insert_metric_name=False,
     save_top_k=1,
    mode="max",
 )
all_callbacks = []
all_callbacks.append(checkpoint_callback)
# train
from pytorch_lightning import seed_everything
seed_everything(123, workers=True)
hm_model = Classifier()
gpus=1
#if torch.cuda.is_available():gpus=0
trainer = pl.Trainer(deterministic=True,max_epochs=60,precision=16,callbacks=all_callbacks)
trainer.fit(hm_model, data_module)

In [ ]:
test_dataloader = DataLoader(dataset=te_p,batch_size=20)
ckpt_path = '/content/LLaVA/mrinal/epoch00-val_f1_all_offn0.00.ckpt' # put ckpt_path according to the path output in the previous cell
trainer.test(dataloaders=test_dataloader,ckpt_path=ckpt_path)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./lightning_logs